In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# The Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/drive/MyDrive/kaggle/Potato/Train"
val_dir = "/content/drive/MyDrive/kaggle/Potato/Valid"

training_images = ImageDataGenerator(rescale=1/255.)
valid_images = ImageDataGenerator(rescale=1/255.)

train_data = training_images.flow_from_directory(train_dir,
                                                 target_size=(256,256),
                                                 color_mode='rgb',
                                                 class_mode='categorical',
                                                 batch_size=32,
                                                 shuffle=True,
                                                 seed=42)
valid_data = valid_images.flow_from_directory(val_dir,
                                              target_size=(256,256),
                                              class_mode='categorical',
                                              seed=42)

Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam

tf.random.set_seed(42)
model = tf.keras.Sequential([
    Conv2D(64, 3, activation='relu', input_shape=(256, 256, 3)),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(16, 3, activation='relu'),
    Conv2D(16, 3, activation='relu'),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
], name="model_leaf_dis")

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(train_data,
                    epochs=10,
                    steps_per_epoch=len(train_data),
                    validation_data=valid_data,
                    validation_steps=len(valid_data))

Epoch 1/10
29/29 [==============================] - 316s 10s/step - loss: 1.1034 - accuracy: 0.4300 - val_loss: 0.9639 - val_accuracy: 0.6233
Epoch 2/10
29/29 [==============================] - 7s 238ms/step - loss: 0.7080 - accuracy: 0.6722 - val_loss: 0.5060 - val_accuracy: 0.7967
Epoch 3/10
29/29 [==============================] - 7s 234ms/step - loss: 0.4683 - accuracy: 0.8078 - val_loss: 0.6278 - val_accuracy: 0.6967
Epoch 4/10
29/29 [==============================] - 7s 232ms/step - loss: 0.3710 - accuracy: 0.8456 - val_loss: 0.3370 - val_accuracy: 0.8500
Epoch 5/10
29/29 [==============================] - 7s 228ms/step - loss: 0.3305 - accuracy: 0.8611 - val_loss: 0.2981 - val_accuracy: 0.8567
Epoch 6/10
29/29 [==============================] - 7s 235ms/step - loss: 0.2619 - accuracy: 0.8956 - val_loss: 0.2016 - val_accuracy: 0.9100
Epoch 7/10
29/29 [==============================] - 7s 226ms/step - loss: 0.2452 - accuracy: 0.9022 - val_loss: 0.1568 - val_accuracy: 0.9600
Epoch 

In [ ]:
test_dir = "/content/drive/MyDrive/kaggle/Potato/Test"
test_images = ImageDataGenerator(rescale=1/255.)

test_data = test_images.flow_from_directory(test_dir,
                                            target_size=(256,256),
                                            seed=42,
                                            class_mode="categorical")

Found 300 images belonging to 3 classes.


In [ ]:
model.evaluate(test_data)

10/10 [==============================] - 60s 7s/step - loss: 0.1712 - accuracy: 0.9467


[0.17124907672405243, 0.9466666579246521]

## Predict

In [ ]:
import numpy as np

In [ ]:
pred = model.predict(test_data)
pred

In [ ]:
pred = np.argmax(pred, axis=1)
pred

In [ ]:
class_counts = np.bincount(pred)

In [ ]:
class_counts

array([104,  88, 108])

In [ ]:
import os

# Replace 'dataset_directory' with the actual path to your dataset directory
dataset_directory = '/content/drive/MyDrive/kaggle/Potato/Train'

# Get a list of subdirectories (class names) in the dataset directory
class_names = np.array(os.listdir(dataset_directory))

print("Class names:", class_names)

Class names: ['Potato___Late_blight' 'Potato___Early_blight' 'Potato___healthy']


## Save the Model

In [ ]:
model.save("leaf_model.h5")